In [2]:
import pandas as pd

In [3]:
from google.colab import files
uploaded = files.upload()

Saving beats_reviews.csv to beats_reviews.csv


In [4]:
df = pd.read_csv('beats_reviews.csv')

In [5]:
print(df.head())

         asin       review_id  \
0  B0D4SX9RC6  R25X0TE59JOMS3   
1  B0D4SX9RC6   RA0QY335MJ1GL   
2  B0D4SX9RC6  R3GFKAF488MHHP   
3  B0D4SX9RC6  R2SW6Y89GDH3FG   
4  B0D4SX9RC6  R3R7XPH8WUAWNW   

                                               title         author  rating  \
0  5.0 out of 5 stars Used Beats Pill 2024 still ...  Leila L’Abate       5   
1  5.0 out of 5 stars So much power in a compact ...         LyneeZ       5   
2                 5.0 out of 5 stars Worth the money  Lauren Bollen       5   
3  5.0 out of 5 stars I'm finding I have to charg...    wanda lyons       5   
4       5.0 out of 5 stars The Kim Gray is excellent         Butter       5   

                                             content  \
0  I bought this used in "acceptable" condition, ...   
1  Just like most others who have reviewed this i...   
2  I’m not going to lie, I wasn’t a skeptic at fi...   
3  Hello. This product is awesome. I listen to mu...   
4  This is a great sounding Bluetooth speaker.

In [10]:
import numpy as np

#Convert to numpy array
ratings = df['rating'].values

#Replace missing values with the mean
mean_rating = np.nanmean(ratings)
ratings = np.where(np.isnan(ratings), mean_rating, ratings)
df['rating'] = ratings

In [11]:
# Keep only valid ratings
valid_ratings = ratings[(ratings >= 0) & (ratings <= 5)]
df = df[df['rating'].isin(valid_ratings)]

[5. 5. 5. 5. 5. 4. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 1. 5. 5. 5. 5.
 4. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 1. 4. 5. 4. 5. 5. 1.]


In [31]:
#Normalize data
min_rating = np.min(ratings)
max_rating = np.max(ratings)
normalized_ratings = (ratings - min_rating) / (max_rating - min_rating)
df['normalized_rating'] = normalized_ratings

In [19]:
#Data types
df['helpful_count'] = df['helpful_count'].astype(np.float64)

In [21]:
#Check for duplicates
print(df.duplicated())

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
dtype: bool


In [22]:
#Review lenghts
review_lengths = np.array([len(content.split()) for content in df['content']])
print(review_lengths)


[550  79  84  54  47   7  41  39 264  56  40  30  43   5   4   1   6  25
  34  83  25 446 339 229  33  39  15   1   4  27  30  38  18  15  11  24
  40  10  51   7   5  45]


In [23]:
#Date
df['clean_date'] = df['timestamp'].str.extract(r'([A-Za-z]+ \d{1,2}, \d{4})')
df['review_date'] = pd.to_datetime(df['clean_date'], format='%B %d, %Y', errors='coerce')
df.drop(columns=['clean_date'], inplace=True)

In [28]:
#Split product attributes
df['color'] = df['product_attributes'].str.extract(r'Color: ([^,]+)')
df['style'] = df['product_attributes'].str.extract(r'Style: ([^,]+)')
df['set'] = df['product_attributes'].str.extract(r'Set: (.+)')

In [33]:
columns_order = ['asin', 'review_id', 'review_date', 'rating', 'normalized_rating', 'title', 'author',
                 'content', 'color', 'style', 'set', 'is_verified', 'helpful_count',
                 'profile_id', 'product_attributes', 'timestamp']
df = df[[col for col in columns_order if col in df.columns]]

In [34]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   asin                42 non-null     object        
 1   review_id           42 non-null     object        
 2   review_date         42 non-null     datetime64[ns]
 3   rating              42 non-null     float64       
 4   normalized_rating   42 non-null     float64       
 5   title               42 non-null     object        
 6   author              42 non-null     object        
 7   content             42 non-null     object        
 8   color               41 non-null     object        
 9   style               21 non-null     object        
 10  set                 13 non-null     object        
 11  is_verified         42 non-null     bool          
 12  helpful_count       42 non-null     float64       
 13  profile_id          27 non-null     object        
 

In [37]:
#Save to csv
np.savetxt('cleaned_reviews.csv', np.column_stack((ratings, review_lengths)), delimiter=",", fmt='%s', header="Rating,Review Length")